![](./logo.png)

In [1]:
import requests
from lxml import etree
import webbrowser
import time
import random as rn
import re
import urllib
import urllib.request
from bs4 import BeautifulSoup
import jieba
import jieba.analyse
import urllib.parse
import json
from hanziconv import HanziConv

In [2]:
歡迎 = ['哈囉！', '嗨！']
回歡迎 = ['怎麼了嗎？', '有什麼事嗎？', '要聊什麼呀？', '怎麼了嗎？']

吃東西 = ['吃', '肚子餓', '餐廳', '小吃', '餓']

喝東西 = ['喝', '口渴', '飲料', '渴']

時間 = ['時間', '幾點', '日期', '幾月', '幾日', '星期']

課程 = ['課程', '課表', '選課']

查詢 = ['谷歌', '查', '你知道', '是什麼', '想知道', '什麼是', 'google']

問選課 = ['下學期課表排好了嗎？', '選課了沒呀？', '要幫忙選課嗎？', '選課有遇到困難嗎？']
回選課 = ['不用', '排好了', '不需要', '不要']

天氣 = ['天氣', '下雨', '太陽', '冷', '熱']

好心情 = ['開心', '快樂', '爽', '欣喜', '愉悅', '愉快']
回好心情 = ['還不錯呀！', '恭喜！', '聽起來不錯', '厲害唷！', '真為你高興！']

壞心情 = ['不開心', '不爽', '糟透了', '不好', '好累', '糟', '煩', '哭哭', '難過', '傷心', '心痛']
回壞心情 = ['拍拍', '一起加油', '沒事拉！', '讓它過去就好了', '別放在心上！']

繼續 = ['要再聊聊嗎？', '多講點', '還有嗎？', '然後呢？', '還有嗎？', '']

感覺 = ['如何', '你呢', '你覺得']
回感覺 = ['還好', '還可以', '尚可', '還可以接受']

感謝 = ['謝', '感謝']
回感謝 = ['不會拉！', '能幫到你很開心！']

離開 = ['掰', '下次見', '下線', '再見', '不聊', '晚安']
回離開 = ['下次見', '下次聊', '掰掰', '很高興認識你！']

敷衍 = ['恩恩', 'XD', '好喔！', '喔！', 'www', 'ＱＱ', '哈哈！', '喔喔！', 'Aっ豆...', '嘻嘻', '好吧！', '呵呵']

刪除關鍵詞庫 =  ["什麼","今天","明天"]

In [3]:
def 問好():
    t = time.localtime(time.time())
    if(t[3]<11 and t[3]>=5):
        return '早安!'
    elif(t[3]>=11 and t[3]<14):
        return '午安!'
    elif(t[3]>=14 and t[3]<16):
        return '下午好!'
    elif(t[3]>=16 and t[3]<23):
        return '晚上好!'
    else:
        return '好晚了!怎麼還不快睡覺!'

In [4]:
# 政大吃什麼
餐點 = {'早午餐':['here cafe', '小公寓', '享窩'],
      '簡餐':['世界大同', '番茄紅', '蝴蝶漾', '葛莉絲', '口甲麵騎士', '金色漁家', '意鼎'],
      '義大利麵':['米塔', '多果', 'go cafe', '里克', 'here cafe', '發現', '享窩', '蝴蝶漾', 'upstair', '鄰居家', '提洛斯'],
      '燉飯':['米塔', '里克', 'here cafe', '發現', '蝴蝶漾', 'upstair', '鄰居家', '提洛斯'],
      '定食':['東京小城', '金鰭', '佐世保', '湯饌', '吉野家', '丼bar', '加賀屋', '樂山', '大正亭'],
      '生魚片':['加賀屋', '大正亭', '東京小城', '金鰭', '佐世保'],
      '越':['華越', '滿福越南料理', '政大越南小吃'],
      '泰':['小曼谷'],
      '炒飯':['敏忠', '廢墟', '波波恰恰', '美香味', '來來', '左撇子'],
      '燴飯':['敏忠', '四五大街', '左撇子', '來來'],
      '炒麵':['廢墟', '敏忠', '來來', '意鼎', '季旭小吃'],
      '乾麵':['京華小館', '甘泉魚麵', 'my麵屋', '口甲麵騎士', '悅來', '飽飽食府', '玉口香', '意鼎', '季旭小吃'],
      '便當':['蘭亭', '扒飯', '美香味', '楊記', '季旭小吃'],
      '麵包':['麵包大亨', '廣奇來', '米塔'],
      '海南雞飯':['私房麵', '波波恰恰'],
      '湯麵':['甘泉魚麵', '京華小館', 'my麵屋', '私房麵', '波波恰恰', '悅來', '飽飽食府', '玉口香', '八方雲集', '吃的小館', '麻咾王拉麵', '金鰭', '東京小城', '飯粥哥', '舅媽的店', '季旭小吃'],
      '東南亞':['華越', '波波恰恰', '私房麵', '滇味廚房', '滿福越南料理', '政大越南小吃'],
      '可麗餅':['屈臣氏旁邊的可麗餅'],
      '飯糰':['傳香', '古早味', '口福'],
      '蛋餅': ['小貓咪', '古早味', '口福', '德合香', '美味派', '戴記', '比司多'],
      '三明治':['享窩', '德合香', '摩斯', '米塔', '口福', '小貓咪', '星巴克', '路易莎', '谷庄茶奶', '美味派', '比司多'],
      '美式':['juicy bun', 'upstair'],
      '合菜':['四川', '巴東蜀味', '敘緣小館', '金色漁家'],
      '滷味':['丐幫', '吃的小館', '健康滷味', '愛心滷味', '京城滷味'],
      '韓':['東萊品', '韓大佬', '阿里郎', '高句麗'],
      '火鍋':['8鍋', '大呼過癮', 'shabu sen'],
      '鍋貼':['八方雲集'],
      '麵線':['展臭豆腐麵線', '舅媽的店', '119巷勝'],
      '臭豆腐':['展臭豆腐麵線'],
      '燒臘':['殺人魔王'],
      '速食':['麥當勞', '摩斯'],
      '素食':['素還真', '愛家蔬食便當'],
      '漢堡':['juicy bun', '麥當勞', '摩斯', '德合香', '小貓咪', 'upstair', '谷庄茶奶', '美味派'],
      '雞排':['政大烤場', '派克'],
      '薯條':['麥當勞', '摩斯', 'juicy bun', 'upstair'],
      '炸雞':['麥當勞', '摩斯'],
      '沙拉':['juicy bun', '摩斯', '麥當勞', '鄰居家', 'upstair', 'subway', 'here cafe'],
      '餅':['東北大餅'],
      '帕尼尼':['here cafe', '小公寓', '路易莎', '鄰居家', '享窩'],
      '蛋捲':['juicy bun', '鄰居家', 'upstair', '路易莎'],
      '潛艇堡':['subway'],
      '水餃':['八方雲集', '悅來', '飽飽食府'],
      '牛肉麵':['飽飽食府', '悅來'],
      '自助餐':['香香', '佳味', '大稻埕'],
      '石鍋拌飯':['高句麗', '韓大佬', '阿里郎', '多果'],
      '咖哩飯':['東京小城', '蝴蝶漾', '119巷勝'],
      '鹹酥雞':['政大烤場'],
      '炸魷魚':['政大烤場'],
      '烤':['政大烤場', '甄燒食堂'],
      '餛飩':['玉口香'],
      '羹':['羹大王', '舅媽的店', '麥側旁的羊肉羹魷魚羹'],
      '滷肉飯':['阿義師', '政大越南小吃', '羹大王', '京華小館', '廢墟', '私房麵', '舅媽的店'],
      '拉麵':['麻咾王拉麵'],
      'pizza':['鄰居家'],
      '粥':['飯粥哥'],
      '壽司':['東京小城', '金鰭', '加賀屋'],
      '貝果':['cow banana'],
      '蛋糕':['米塔', '廣奇來', '麵包大亨', '路易莎', '星巴克', '5cafe', 'joline'],
      '豆花':['明池'],
      '西米露':['明池'],
      '仙草':['明池', '廢墟前的台灣傳統客家'],
      '咖啡店':['星巴克', '路易莎', '道南館', 'prowin paris coffee', 'lee brothers 44 bisto', '5cafe'],
      '水煎包':['老麵水煎包']}
飲料 = {'奶茶':['龍角', '五十嵐', 'Comebuy', '政大茶亭', '星巴克', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶'],
      '咖啡':['龍角', '五十嵐', 'Comebuy', '星巴克', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶', '5 Cafe'],
      '鮮奶茶':['龍角', 'Cow banana', '找茶道', '谷庄茶奶'],
      '果茶':['龍角', 'Comebuy', '政大茶亭', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶'],
      '果汁':['Comebuy', '政大茶亭', '星巴克', '咖啡大亨'],
      '茶':['龍角', '五十嵐', 'Comebuy', '路易莎', 'Cow banana', '咖啡大亨', '找茶道', '谷庄茶奶', '5 Cafe'],
      '豆奶':['5 Cafe'],
      '泡沫':['Comebuy', '政大茶亭', '咖啡大亨'],
      '星冰樂':['星巴克'],
      '啤酒':['5 Cafe'],
      '冰品':['龍角', 'Comebuy', '政大茶亭', '星巴克', '找茶道'],
      '明星':['五十嵐', 'Comebuy', '星巴克', '找茶道'],
      '新品':['五十嵐', 'Comebuy', '星巴克', '找茶道']}

決定 = ['好', '決定', '不錯', '勉強', '可以']
回決定 = ['太棒了' ,'幫到你很開心', '用餐愉快！']

詢問 = ['如何呢？', '可以嗎？', 'ＯＫ嗎？', '可以接受嗎？', '要嗎？']

# 沒選擇 = ['隨便', '沒想法', '都可以', '推薦', '建議']

否定 = ['不要', '換', '沒']
建議 = ['有特別想要什麼嗎？', '想要什麼種類呢？', '舉個例子吧？', '有特別喜歡什麼嗎？']

def 菜單中(message, food):
    for i in food.keys():
        if i in message:
            return i
        
def 食物(name, food):
    message = ''
    while not any(i in message for i in 決定):
        print('%s: %s' %('Aっ豆', rn.choice(建議)))
        message = input('%s: ' %(name))
        if 菜單中(message, food):
            print('%s: 可以參考 %s, %s' %('Aっ豆', rn.choice(food[菜單中(message,food)]), rn.choice(詢問)))
        elif (not 菜單中(message, food)) and (not any(i in message for i in 否定)) :
            print('%s: 好像沒有耶！要不要參考 %s呢？' %('Aっ豆', rn.choice(food[rn.choice(list(food.keys()))])))
        else:
            print('%s: %s %s' %('Aっ豆', rn.choice(food[rn.choice(list(food.keys()))]), rn.choice(詢問)))
        message = input('%s: ' %(name))
    print('%s: %s %s' %('Aっ豆', rn.choice(回決定), rn.choice(繼續)))

In [5]:
# 天氣
def weather(name):
    url = "https://tw.news.yahoo.com/weather/%E8%87%BA%E7%81%A3/%E8%87%BA%E5%8C%97%E5%B8%82/%E8%87%BA%E5%8C%97%E5%B8%82-2306179"
    response = requests.get(url)
    html = response.text
    page = etree.HTML(html)
    print('%s: 查詢中。。。' %('Aっ豆'))
    print('%s: 這是我幫你查的天氣！' %('Aっ豆'))
    lo=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[1]/div[1]/div/h1//text()')
    t=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[3]/span[1]//text()')
    h=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[2]/span[1]//text()')
    l=page.xpath('//*[@id="Lead-1-WeatherLocationAndTemperature"]/div/section[2]/div/div[2]/span[2]//text()')
    r=page.xpath('//*[@id="weather-precipitation"]/div/div[4]/span/span//text()')
    print('         現在溫度 %s : %s 度' %(lo[0],t[0]))
    print('         今日最高溫為 %s 度' %h[0])
    print('         今日最低溫為 %s 度' %l[0])
    print('         今日下雨機率 : %s%s' %(r[0],r[1]))
    p=int(r[0])
    if(p < 20):
        print('        天氣不錯呢!')
    elif (20 < p < 60):
        print('         可能會下雨喔!')
    else:
        print('         下雨機率有點高ㄟ!記得出門要帶傘喔!!')
    print('%s: 要進入網頁嗎?' %('Aっ豆'))
    y = input('%s: ' %(name))
    if("不" not in y):
        a = webbrowser.get(''+webbrowser._tryorder[0]+'')
        a.open("https://tw.news.yahoo.com/weather/%E8%87%BA%E7%81%A3/%E8%87%BA%E5%8C%97%E5%B8%82/%E8%87%BA%E5%8C%97%E5%B8%82-2306179")
    print("%s :知道天氣就可以安心出門囉!" %('Aっ豆'))

In [6]:
# 翻譯
def translate(from_lang, to_lang, content):   
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=http://www.youdao.com/'
    data = {'type':'',
            'i':content,
            'doctype':'json',
            'xmlVersion':'1.8',
            'keyfrom':'fanyi.web',
            'ue':'UTF-8',
            'action':'FY_BY_CLICKBUTTON',
            'typoResult':'true'}
    
    if from_lang == 'ZH_CN' or to_lang == 'ZH_CN':
        data['type'] = from_lang + '2' + to_lang
        data = urllib.parse.urlencode(data).encode('utf-8')
        response = urllib.request.urlopen(url, data)
        html = response.read().decode('utf-8')
        target = json.loads(html)
        if to_lang == 'ZH_CN':
            return HanziConv.toTraditional(target['translateResult'][0][0]['tgt'])
        else:
            return target['translateResult'][0][0]['tgt']
    else:
        data['type'] = from_lang + '2' + 'ZH_CN'
        data_1 = urllib.parse.urlencode(data).encode('utf-8')
        response = urllib.request.urlopen(url, data_1)  
        html = response.read().decode('utf-8')
        target = json.loads(html)
        
        data['type'] = 'ZH_CN' + '2' + to_lang
        data['i'] = target['translateResult'][0][0]['tgt']
        data2 = urllib.parse.urlencode(data).encode('utf-8')
        response = urllib.request.urlopen(url, data2)  
        html = response.read().decode('utf-8')
        target = json.loads(html)
        return target['translateResult'][0][0]['tgt']

語言 = {'英文':'EN','中文':'ZH_CN', '日文':'JA', '韓文':'KR', '法文':'FR', '俄文':'RU', '西班牙文':'SP', '葡萄牙文':'PT'}

def 翻譯(name):
    conti = ''
    while '不' not in conti:    
        print('%s: 你想翻譯哪兩國語言呢？' %('Aっ豆'))
        print('       若想將中文翻譯成英文，請輸入：中文，英文')
        language = input('%s:' %(name)).split('，')
        while (language[0] not in 語言) or (language[1] not in 語言) or language[0] == language[1]:
            if (language[0] not in 語言):
                print('%s: 不好意思， 尚無法處理 %s' %('Aっ豆', language[0]))
            elif (language[1] not in 語言):
                print('%s: 不好意思， 尚無法處理 %s' %('Aっ豆', language[1]))
            else:
                print('%s: 兩個一樣囉！' %('Aっ豆'))
            print('%s: 你想翻譯哪兩國語言呢？' %('Aっ豆'))
            print('       若想將中文翻譯成英文，請輸入：中文，英文')
            language = input('%s:' %(name)).split('，')
        print('%s: 你想要翻譯什麼？' %('Aっ豆'))
        text = input('%s:' %(name))
        print('%s: 翻譯中。。。' %('Aっ豆'))
        print('%s: %s' %('Aっ豆', translate(語言[language[0]], 語言[language[1]], text)))
        print('%s: 還要繼續嗎？' %('Aっ豆'))
        conti = input('%s: ' %(name))
    print('%s: 很開心幫到你！' %('Aっ豆'))

In [7]:
# 基本查詢課程
def searchCourse(課程名稱, 指導教授):
    resq = requests.Session() # 雖然名稱同樣叫Session，但是這是 requests 的物件，是專門用來保持網路間連線的！
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'Host': 'wa.nccu.edu.tw',
        'Referer': 'http://wa.nccu.edu.tw/QryTor/',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    
    
    class MyOpener(urllib.request.URLopener):
        version = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0'

    myopener = MyOpener()
    url = 'http://wa.nccu.edu.tw/QryTor/'
    # first HTTP request without form data
    f = myopener.open(url)
    soup = BeautifulSoup(f, "lxml")
    # parse and retrieve two vital form values
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    eventvalidation = soup.select("#__EVENTVALIDATION")[0]['value']
    
    post_data = {
        "__EVENTARGUMENT": "",
        "__EVENTTARGET": "searchA",
        "__EVENTVALIDATION": eventvalidation,
        "__LASTFOCUS": "",
        "__SCROLLPOSITIONX":"0",
        "__SCROLLPOSITIONY":"0",
        "__VIEWSTATE": viewstate,
        "__VIEWSTATEGENERATOR": "",
        "coursenameTB": 課程名稱,
        "instructorTB": 指導教授,
        "lang": "1",
        "languageCBL$0": "on",
        "languageCBL$1": "on",
        "languageCBL$2":"on",
        "yyssDDL": "1052"
    }
    resp = resq.post("http://wa.nccu.edu.tw/QryTor/",data=post_data)
    html = resp.text
    page = etree.HTML(html)

    postListnum = re.compile(r'[^\d.]+').sub('', page.xpath("//span[@id='countL']//text()")[0].split(",")[0])
    if int(postListnum) > 20:
        postListnum = 20
        print("Aっ豆: ㄟ豆...因為覺得列太多資料太麻煩，所以只列前%2d筆資料喔！"%(int(postListnum)))
    else:
        print("Aっ豆: ㄟ豆...幫你找到這學期你查的課程只有%2d筆資料喔!"%(int(postListnum)))
    for i in range(int(postListnum)):
        print(page.xpath("//a[@id='qryresult"+str(i)+"_course_nameHL']//text()")[0].split("/")[0],page.xpath("//a[@id='qryresult"+str(i)+"_instructorHL']//text()")[0].split("/")[0],page.xpath("//span[@id='qryresult"+str(i)+"_sessionL']//text()")[0].split("/")[0],"科目代碼是：",page.xpath("//span[@id='qryresult"+str(i)+"_coursenumL']//text()")[0].split("/")[0])

        
# 進階查詢課程
def 進階查詢(星期,課程代碼,課程名稱,指導教授):
    星期表 = {
        "一":"一", "二":"二", "三":"三", "四":"四", "五":"五", "六":"六", "日":"日","天":"日",
        "1":"一", "2":"二", "3":"三", "4":"四", "5":"五", "6":"六", "7":"日" 
         }

    星期一 = 星期二 = 星期三 = 星期四 = 星期五 = 星期六 = 星期日 = "off"
    查詢星期輸入 = list(星期) # 這邊是input
    查詢星期 = []
    for i in 查詢星期輸入:
        if i in 星期表:
            查詢星期.append(星期表[i])
    if '一' in 查詢星期:星期一 = ""
    if '二' in 查詢星期:星期二 = ""
    if '三' in 查詢星期:星期三 = ""
    if '四' in 查詢星期:星期四 = ""
    if '五' in 查詢星期:星期五 = ""
    if '六' in 查詢星期:星期六 = ""
    if '日' in 查詢星期:星期日 = ""

    resq = requests.Session()
    cookies = {
        '_ga': 'GA1.3.946599470.1481424474',
        'ASP.NET_SessionId': 'n3avfymhzr2f42g4rgnibaog',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'Host': 'wa.nccu.edu.tw',
        'Referer': 'http://wa.nccu.edu.tw/QryTor/Default.aspx?language=zh-TW',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0',
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    data = {
       '__EVENTTARGET': 'searchA',
      '__EVENTARGUMENT': '',
      '__LASTFOCUS': '',
      '__VIEWSTATE': '/wEPDwUKMTgxMDQwNzAxNQ9kFgICAQ9kFgICAQ9kFhgCCw8WAh4EVGV4dAVC6Kqy56iL57Si5byV6LOH5paZ5qqU5pyA5b6M5pu05paw5pmC6ZaT54K6MjAxNy8xLzgg5LiK5Y2IIDEyOjAwOjAwZAITDw8WAh8ABQ/lm57ln7rnpI7mn6XoqaJkZAIXDxAPFgIeC18hRGF0YUJvdW5kZ2QQFSEG5YWo6YOoLDEwNeWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMTcgU3ByaW5nIFNlbWVzdGVyKjEwNeWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMTYgRmFsbCBTZW1lc3RlciwxMDTlrbjlubTluqbnrKwy5a245pyfICAyMDE2IFNwcmluZyBTZW1lc3RlcioxMDTlrbjlubTluqbnrKwx5a245pyfICAyMDE1IEZhbGwgU2VtZXN0ZXIsMTAz5a245bm05bqm56ysMuWtuOacnyAgMjAxNSBTcHJpbmcgU2VtZXN0ZXIqMTAz5a245bm05bqm56ysMeWtuOacnyAgMjAxNCBGYWxsIFNlbWVzdGVyLDEwMuWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMTQgU3ByaW5nIFNlbWVzdGVyKjEwMuWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMTMgRmFsbCBTZW1lc3RlciwxMDHlrbjlubTluqbnrKwy5a245pyfICAyMDEzIFNwcmluZyBTZW1lc3RlcioxMDHlrbjlubTluqbnrKwx5a245pyfICAyMDEyIEZhbGwgU2VtZXN0ZXIsMTAw5a245bm05bqm56ysMuWtuOacnyAgMjAxMiBTcHJpbmcgU2VtZXN0ZXIqMTAw5a245bm05bqm56ysMeWtuOacnyAgMjAxMSBGYWxsIFNlbWVzdGVyKzk55a245bm05bqm56ysMuWtuOacnyAgMjAxMSBTcHJpbmcgU2VtZXN0ZXIpOTnlrbjlubTluqbnrKwx5a245pyfICAyMDEwIEZhbGwgU2VtZXN0ZXIrOTjlrbjlubTluqbnrKwy5a245pyfICAyMDEwIFNwcmluZyBTZW1lc3Rlcik5OOWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDkgRmFsbCBTZW1lc3Rlcis5N+WtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDkgU3ByaW5nIFNlbWVzdGVyKTk35a245bm05bqm56ysMeWtuOacnyAgMjAwOCBGYWxsIFNlbWVzdGVyKzk25a245bm05bqm56ysMuWtuOacnyAgMjAwOCBTcHJpbmcgU2VtZXN0ZXIpOTblrbjlubTluqbnrKwx5a245pyfICAyMDA3IEZhbGwgU2VtZXN0ZXIrOTXlrbjlubTluqbnrKwy5a245pyfICAyMDA3IFNwcmluZyBTZW1lc3Rlcik5NeWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDYgRmFsbCBTZW1lc3Rlcis5NOWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDYgU3ByaW5nIFNlbWVzdGVyKTk05a245bm05bqm56ysMeWtuOacnyAgMjAwNSBGYWxsIFNlbWVzdGVyKzkz5a245bm05bqm56ysMuWtuOacnyAgMjAwNSBTcHJpbmcgU2VtZXN0ZXIpOTPlrbjlubTluqbnrKwx5a245pyfICAyMDA0IEZhbGwgU2VtZXN0ZXIrOTLlrbjlubTluqbnrKwy5a245pyfICAyMDA0IFNwcmluZyBTZW1lc3Rlcik5MuWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDMgRmFsbCBTZW1lc3Rlcis5MeWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDMgU3ByaW5nIFNlbWVzdGVyKTkx5a245bm05bqm56ysMeWtuOacnyAgMjAwMiBGYWxsIFNlbWVzdGVyKzkw5a245bm05bqm56ysMuWtuOacnyAgMjAwMiBTcHJpbmcgU2VtZXN0ZXIpOTDlrbjlubTluqbnrKwx5a245pyfICAyMDAxIEZhbGwgU2VtZXN0ZXIVIQAEMTA1MgQxMDUxBDEwNDIEMTA0MQQxMDMyBDEwMzEEMTAyMgQxMDIxBDEwMTIEMTAxMQQxMDAyBDEwMDEEMDk5MgQwOTkxBDA5ODIEMDk4MQQwOTcyBDA5NzEEMDk2MgQwOTYxBDA5NTIEMDk1MQQwOTQyBDA5NDEEMDkzMgQwOTMxBDA5MjIEMDkyMQQwOTEyBDA5MTEEMDkwMgQwOTAxFCsDIWdnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2RkAisPDxYCHgdWaXNpYmxlZ2RkAi0PFgIeBXN0eWxlBRliYWNrZ3JvdW5kLWNvbG9yOiNFQkVCRUI7ZAIvDxYCHwMFGWJhY2tncm91bmQtY29sb3I6I0Y1RjVGNTtkAjEPDxYCHwJnZGQCNw8QZGQWAGQCOQ9kFgICAw8QZGQWAGQCOw9kFgICAw8QZGQWAGQCRw8PFgIfAmdkZAJNDw8WAh8ABQ/lm57ln7rnpI7mn6XoqaJkZBgBBR5fX0NvbnRyb2xzUmVxdWlyZVBvc3RCYWNrS2V5X18WIQUNbGFuZ3VhZ2VDQkwkMAUNbGFuZ3VhZ2VDQkwkMQUNbGFuZ3VhZ2VDQkwkMgUNbGFuZ3VhZ2VDQkwkMgUMc2Vzc2lvbkNCTCQwBQxzZXNzaW9uQ0JMJDEFDHNlc3Npb25DQkwkMgUMc2Vzc2lvbkNCTCQzBQxzZXNzaW9uQ0JMJDQFDHNlc3Npb25DQkwkNQUMc2Vzc2lvbkNCTCQ2BQxzZXNzaW9uQ0JMJDYFDHNlY3Rpb25DQkwkMAUMc2VjdGlvbkNCTCQxBQxzZWN0aW9uQ0JMJDIFDHNlY3Rpb25DQkwkMwUMc2VjdGlvbkNCTCQzBQtzdGF0dXNDQkwkMAULc3RhdHVzQ0JMJDEFC3N0YXR1c0NCTCQyBQtzdGF0dXNDQkwkMgUIZmFyQ0JMJDAFCGZhckNCTCQxBQhmYXJDQkwkMQUJa2VybENCTCQwBQlrZXJsQ0JMJDEFCWtlcmxDQkwkMQUNZWxpZ2libGVDQkwkMAUNZWxpZ2libGVDQkwkMQUNZWxpZ2libGVDQkwkMQULY2hhcmdlQ0JMJDAFC2NoYXJnZUNCTCQxBQtjaGFyZ2VDQkwkMYrn8MjCebw71js1pC7GTQD7Wb6H2dNG9+EtEYaudAML',
      '__VIEWSTATEGENERATOR': '8D42D28B',
      '__SCROLLPOSITIONX': '0',
      '__SCROLLPOSITIONY': '1753',
      '__EVENTVALIDATION': '/wEdAF+WQooGfHJ4FTISStUcdponrzrITmbATvf5Cnrc0eB2tGxjLm6vgBlLdqcpvDjBzSYemxKP2fAGjeArYsX2X0YrHrrYirCU719niJ6FjXied2nj04ol9o4AH9XeFHrZxktPD3djkYpv2Wt518bEbFAYvPhNPRtGMGIPXXymggVhTdzK1dVvw/utHEKIo6EKH65N3eZmzuIl1HFqNAbBUs7R4Ru1C3gYP59WSd1uJwbCFGxOM4lwkmlwSNNfJFHwbYzKmxiVjK6LQt9CGa2qJlXbZ1TLcusKO3RX8wsFQ6uKZHhl+yHOEyKBFsSzVF6DTe6BD9Bw2yjv5F/WFcu8vX1VrN4HUOvf1fLu4Uf1X2D7lTH/xYDMKs7BHDFOtldtPnnFCQJeAL3tJjlNiPPX0JfsyW2xD2MiOpD0hfJ77exmDun+UFVzMw+nPoPWrE48jKFdWoeGZIry+KY9k6i8OVExgrnVWup3GZBLaq5a2XOMJz5QD2v9jD/x+gY4GamTPVBQemezvvhLkapW9IuxjYmoxtcZttEgcOZnhFNnMwgddfxTkdWmsVNGW4/65tu63pZ7VA/oXngCQf4G+sIN4quAdQr0GCjdVjBR86uVMRRu9Vp26ICNp3RUdI4S3SpUdL0byLwsTRfk7hhoMrFUin6i7p4i1MiR95t1Irp+0aNrs+49jHwyYTUH4LT9fwdQXc6mz7PQm8iuf0IouHKpG6vwqfTaW93RypgcLUWUVHKTQoLUMVKHV9NEhdarMajyPoAqsDETbwAvwZ+3MWyDbplBLCHZQYvOZON7jczfoyeXji50m5PlzrYiLlCEk6HUnJ1RrJ2q8ytjz79LjIyxnbQ3sQzWr+kqSOQfuY95AUhu2dDC/jvtH7zabLo0Ipu8en3Q0pnfniLl5SYbpMW6yfVwgVAVRTorh5pyzzeeXPtgK8fW3GCUaR5ZtEhI/l2b5joqWiCkVM2eNBMql5gvk9tPZ+ggVWJkm91svxs+327A5lTyOYN9DxQyS5YI+Ssa6oExBpTSmoZwyhBj9evubd2Kxu/GSKR7BwUIgstOgODX78Z3IKLHvY/ywL/4hiH/KuwsNTK2zhUZKg8p0Ty2nts5T1wk/O2gGHPQfd7nPUQlaC09b+eOj9Kd7NOoye4g55U8+beog/B1cFFeQh64ZX9HnxueqVeq/2x+bPFhY/V9uJ5OEpNq6TYSz1JrM4CajNfh+qsW++MBfIWw846v4N5EWkwA6Rc3qV4uRNppyhlLHKBjJcvdGWtVJcyr2za9NGM3T380IFlbGqNKjT62mTrzsTEVCPBvimfKSTU0fPc2Q3+kDsdqsYvos2WMjYc71fIBVxIE31RtleapjWGbaGqo7Ipbt+ATS6CCcYHVj0uV5XnfLNv8QLp7knY3ttGKnfSQbtwC1w1RacvoIKcYyi96DdPtsg+5oV5/leCQRsHqnCDJqUen4SsBlUGrlxNI7Y9eHdAtAjGGm2L34IFQfHwnzHIXWwyslXNCljg5XKGepmgsQgCF1fwSS/Me180GSZSQTtnHXQGfPVgyKN1/lWeV4bNnt+B++l91hImQqGlBUG/hlc2mgbZd5o9esYyq87f9Y59YAkbpFnJvdzqvW6ZUuUMR3L3LAWzrO3YHTx3sEYnz+v1fIlQKdGxxLBtNQn0HcZgGIqc/yNbPyDChju5PbET9VnXphW48ioYyPUOXE/TqsYgEGqDYuAkgNn9x6qJ36nG7rxXsci3YpDbM5h4Rgvh35U6aVIovYt/7ESJD9VMEJIohqIXs7kJOMookXvpMDIR/KMoIQiK9YTnwSKTst+zSAvU8KCQyiqlS/ihXFCkU/3KsCnYK615gVSGtX+3Oiyc+RvhA10klkY+3yCkPbO6+CGoVjCb5iCd7DIeoU0a1Tf0HGZUrFry1QX6kdiVDAME0+8Nsd5Tg9bYFvJ0MGaJMyhxKkUduxaGH9tlxiA3XYwi11ebaur0x7bxnhaa9NrIR2vPpyMmxmRUsp+UZ4SyDGmERAWS616z/t0v2+/sr0/xshJJnLTdxEphT1+32kXvX//m/wLqubtkWcLze',
      'yyssDDL': '1052',
      'languageCBL$0': 'on',
      'languageCBL$1': 'on',
      'languageCBL$2': 'on',
      'courseTB': 課程代碼,
      'coursenameTB': 課程名稱,
      'instructorTB': 指導教授,
      'sessionTB': '',
      'sessionCBL'+星期一+'$0': 'on',
      'sessionCBL'+星期二+'$1': 'on',
      'sessionCBL'+星期三+'$2': 'on',
      'sessionCBL'+星期四+'$3': 'on',
      'sessionCBL'+星期五+'$4': 'on',
      'sessionCBL'+星期六+'$5': 'on',
      'sessionCBL'+星期日+'$6': 'on',
      'sectionCBL$0': 'on',
      'sectionCBL$1': 'on',
      'sectionCBL$2': 'on',
      'sectionCBL$3': 'on',
      'statusCBL$0': 'on',
      'statusCBL$1': 'on',
      'statusCBL$2': 'on',
      'farCBL$0': 'on',
      'farCBL$1': 'on',
      'kerlCBL$0': 'on',
      'kerlCBL$1': 'on',
      'eligibleCBL$0': 'on',
      'eligibleCBL$1': 'on',
      'chargeCBL$0': 'on',
      'chargeCBL$1': 'on',
      'remainRBL': 'all',
      'numberpageRBL': '50',
      'lang': '1'
    }

    resp = resq.post('http://wa.nccu.edu.tw/QryTor/Default.aspx?language=zh-TW', data=data)
    html = resp.text
    page = etree.HTML(html)
    try:
        postListnum = re.compile(r'[^\d.]+').sub('', page.xpath("//span[@id='countL']//text()")[0].split(",")[0])
        if int(postListnum) > 50:
            postListnum = 50
            print("ㄟ豆...因為覺得列太多資料太麻煩，所以只列前%2d筆資料喔!"%int(postListnum))
        else:
            print("ㄟ豆...幫你找到這學期你查的課程只有%2d筆資料喔!"%int(postListnum))
        for i in range(int(postListnum)):
            print(page.xpath("//a[@id='qryresult"+str(i)+"_course_nameHL']//text()")[0].split("/")[0],page.xpath("//a[@id='qryresult"+str(i)+"_instructorHL']//text()")[0].split("/")[0],page.xpath("//span[@id='qryresult"+str(i)+"_sessionL']//text()")[0].split("/")[0],"科目代碼是：",page.xpath("//span[@id='qryresult"+str(i)+"_coursenumL']//text()")[0].split("/")[0])
    except:
        print("ㄟ豆..."+page.xpath("//span[@id='description']//text()")[0])
        
# 加入清單
def 加入課程(使用者帳號, 使用者密碼, 課程代碼):
    if 使用者帳號 == "":
        print("誒豆...你的帳號空白了呀...")
    elif 使用者密碼 == "":
        print("誒豆...你的密碼空白了呀...")
    elif (課程代碼 == "") or (len(課程代碼)) != 9 or (課程代碼.isdigit() != True):
        print("誒豆...至少告訴我正確的代碼呀...")
    else:
        resq = requests.Session()
        class MyOpener(urllib.request.URLopener):
            version = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0'

        myopener = MyOpener()
        url = 'http://wa.nccu.edu.tw/QryTor/goMySelSubList.aspx?course=6a5264645475362f7451557168433661525174345a513d3d'
        f = myopener.open(url)
        soup = BeautifulSoup(f, "lxml")
        viewstate = soup.select("#__VIEWSTATE")[0]['value']
        
        data = {
          '__EVENTTARGET': 'searchA',
          '__EVENTARGUMENT': '',
          '__LASTFOCUS': '',
          '__VIEWSTATE': '/wEPDwUKMTgxMDQwNzAxNQ9kFgICAQ9kFgICAQ9kFhgCCw8WAh4EVGV4dAVC6Kqy56iL57Si5byV6LOH5paZ5qqU5pyA5b6M5pu05paw5pmC6ZaT54K6MjAxNy8xLzgg5LiK5Y2IIDEyOjAwOjAwZAITDw8WAh8ABQ/lm57ln7rnpI7mn6XoqaJkZAIXDxAPFgIeC18hRGF0YUJvdW5kZ2QQFSEG5YWo6YOoLDEwNeWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMTcgU3ByaW5nIFNlbWVzdGVyKjEwNeWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMTYgRmFsbCBTZW1lc3RlciwxMDTlrbjlubTluqbnrKwy5a245pyfICAyMDE2IFNwcmluZyBTZW1lc3RlcioxMDTlrbjlubTluqbnrKwx5a245pyfICAyMDE1IEZhbGwgU2VtZXN0ZXIsMTAz5a245bm05bqm56ysMuWtuOacnyAgMjAxNSBTcHJpbmcgU2VtZXN0ZXIqMTAz5a245bm05bqm56ysMeWtuOacnyAgMjAxNCBGYWxsIFNlbWVzdGVyLDEwMuWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMTQgU3ByaW5nIFNlbWVzdGVyKjEwMuWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMTMgRmFsbCBTZW1lc3RlciwxMDHlrbjlubTluqbnrKwy5a245pyfICAyMDEzIFNwcmluZyBTZW1lc3RlcioxMDHlrbjlubTluqbnrKwx5a245pyfICAyMDEyIEZhbGwgU2VtZXN0ZXIsMTAw5a245bm05bqm56ysMuWtuOacnyAgMjAxMiBTcHJpbmcgU2VtZXN0ZXIqMTAw5a245bm05bqm56ysMeWtuOacnyAgMjAxMSBGYWxsIFNlbWVzdGVyKzk55a245bm05bqm56ysMuWtuOacnyAgMjAxMSBTcHJpbmcgU2VtZXN0ZXIpOTnlrbjlubTluqbnrKwx5a245pyfICAyMDEwIEZhbGwgU2VtZXN0ZXIrOTjlrbjlubTluqbnrKwy5a245pyfICAyMDEwIFNwcmluZyBTZW1lc3Rlcik5OOWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDkgRmFsbCBTZW1lc3Rlcis5N+WtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDkgU3ByaW5nIFNlbWVzdGVyKTk35a245bm05bqm56ysMeWtuOacnyAgMjAwOCBGYWxsIFNlbWVzdGVyKzk25a245bm05bqm56ysMuWtuOacnyAgMjAwOCBTcHJpbmcgU2VtZXN0ZXIpOTblrbjlubTluqbnrKwx5a245pyfICAyMDA3IEZhbGwgU2VtZXN0ZXIrOTXlrbjlubTluqbnrKwy5a245pyfICAyMDA3IFNwcmluZyBTZW1lc3Rlcik5NeWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDYgRmFsbCBTZW1lc3Rlcis5NOWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDYgU3ByaW5nIFNlbWVzdGVyKTk05a245bm05bqm56ysMeWtuOacnyAgMjAwNSBGYWxsIFNlbWVzdGVyKzkz5a245bm05bqm56ysMuWtuOacnyAgMjAwNSBTcHJpbmcgU2VtZXN0ZXIpOTPlrbjlubTluqbnrKwx5a245pyfICAyMDA0IEZhbGwgU2VtZXN0ZXIrOTLlrbjlubTluqbnrKwy5a245pyfICAyMDA0IFNwcmluZyBTZW1lc3Rlcik5MuWtuOW5tOW6puesrDHlrbjmnJ8gIDIwMDMgRmFsbCBTZW1lc3Rlcis5MeWtuOW5tOW6puesrDLlrbjmnJ8gIDIwMDMgU3ByaW5nIFNlbWVzdGVyKTkx5a245bm05bqm56ysMeWtuOacnyAgMjAwMiBGYWxsIFNlbWVzdGVyKzkw5a245bm05bqm56ysMuWtuOacnyAgMjAwMiBTcHJpbmcgU2VtZXN0ZXIpOTDlrbjlubTluqbnrKwx5a245pyfICAyMDAxIEZhbGwgU2VtZXN0ZXIVIQAEMTA1MgQxMDUxBDEwNDIEMTA0MQQxMDMyBDEwMzEEMTAyMgQxMDIxBDEwMTIEMTAxMQQxMDAyBDEwMDEEMDk5MgQwOTkxBDA5ODIEMDk4MQQwOTcyBDA5NzEEMDk2MgQwOTYxBDA5NTIEMDk1MQQwOTQyBDA5NDEEMDkzMgQwOTMxBDA5MjIEMDkyMQQwOTEyBDA5MTEEMDkwMgQwOTAxFCsDIWdnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2RkAisPDxYCHgdWaXNpYmxlZ2RkAi0PFgIeBXN0eWxlBRliYWNrZ3JvdW5kLWNvbG9yOiNFQkVCRUI7ZAIvDxYCHwMFGWJhY2tncm91bmQtY29sb3I6I0Y1RjVGNTtkAjEPDxYCHwJnZGQCNw8QZGQWAGQCOQ9kFgICAw8QZGQWAGQCOw9kFgICAw8QZGQWAGQCRw8PFgIfAmdkZAJNDw8WAh8ABQ/lm57ln7rnpI7mn6XoqaJkZBgBBR5fX0NvbnRyb2xzUmVxdWlyZVBvc3RCYWNrS2V5X18WIQUNbGFuZ3VhZ2VDQkwkMAUNbGFuZ3VhZ2VDQkwkMQUNbGFuZ3VhZ2VDQkwkMgUNbGFuZ3VhZ2VDQkwkMgUMc2Vzc2lvbkNCTCQwBQxzZXNzaW9uQ0JMJDEFDHNlc3Npb25DQkwkMgUMc2Vzc2lvbkNCTCQzBQxzZXNzaW9uQ0JMJDQFDHNlc3Npb25DQkwkNQUMc2Vzc2lvbkNCTCQ2BQxzZXNzaW9uQ0JMJDYFDHNlY3Rpb25DQkwkMAUMc2VjdGlvbkNCTCQxBQxzZWN0aW9uQ0JMJDIFDHNlY3Rpb25DQkwkMwUMc2VjdGlvbkNCTCQzBQtzdGF0dXNDQkwkMAULc3RhdHVzQ0JMJDEFC3N0YXR1c0NCTCQyBQtzdGF0dXNDQkwkMgUIZmFyQ0JMJDAFCGZhckNCTCQxBQhmYXJDQkwkMQUJa2VybENCTCQwBQlrZXJsQ0JMJDEFCWtlcmxDQkwkMQUNZWxpZ2libGVDQkwkMAUNZWxpZ2libGVDQkwkMQUNZWxpZ2libGVDQkwkMQULY2hhcmdlQ0JMJDAFC2NoYXJnZUNCTCQxBQtjaGFyZ2VDQkwkMYrn8MjCebw71js1pC7GTQD7Wb6H2dNG9+EtEYaudAML',
          '__VIEWSTATEGENERATOR': '8D42D28B',
          '__SCROLLPOSITIONX': '0',
          '__SCROLLPOSITIONY': '1753',
          '__EVENTVALIDATION': '/wEdAF+WQooGfHJ4FTISStUcdponrzrITmbATvf5Cnrc0eB2tGxjLm6vgBlLdqcpvDjBzSYemxKP2fAGjeArYsX2X0YrHrrYirCU719niJ6FjXied2nj04ol9o4AH9XeFHrZxktPD3djkYpv2Wt518bEbFAYvPhNPRtGMGIPXXymggVhTdzK1dVvw/utHEKIo6EKH65N3eZmzuIl1HFqNAbBUs7R4Ru1C3gYP59WSd1uJwbCFGxOM4lwkmlwSNNfJFHwbYzKmxiVjK6LQt9CGa2qJlXbZ1TLcusKO3RX8wsFQ6uKZHhl+yHOEyKBFsSzVF6DTe6BD9Bw2yjv5F/WFcu8vX1VrN4HUOvf1fLu4Uf1X2D7lTH/xYDMKs7BHDFOtldtPnnFCQJeAL3tJjlNiPPX0JfsyW2xD2MiOpD0hfJ77exmDun+UFVzMw+nPoPWrE48jKFdWoeGZIry+KY9k6i8OVExgrnVWup3GZBLaq5a2XOMJz5QD2v9jD/x+gY4GamTPVBQemezvvhLkapW9IuxjYmoxtcZttEgcOZnhFNnMwgddfxTkdWmsVNGW4/65tu63pZ7VA/oXngCQf4G+sIN4quAdQr0GCjdVjBR86uVMRRu9Vp26ICNp3RUdI4S3SpUdL0byLwsTRfk7hhoMrFUin6i7p4i1MiR95t1Irp+0aNrs+49jHwyYTUH4LT9fwdQXc6mz7PQm8iuf0IouHKpG6vwqfTaW93RypgcLUWUVHKTQoLUMVKHV9NEhdarMajyPoAqsDETbwAvwZ+3MWyDbplBLCHZQYvOZON7jczfoyeXji50m5PlzrYiLlCEk6HUnJ1RrJ2q8ytjz79LjIyxnbQ3sQzWr+kqSOQfuY95AUhu2dDC/jvtH7zabLo0Ipu8en3Q0pnfniLl5SYbpMW6yfVwgVAVRTorh5pyzzeeXPtgK8fW3GCUaR5ZtEhI/l2b5joqWiCkVM2eNBMql5gvk9tPZ+ggVWJkm91svxs+327A5lTyOYN9DxQyS5YI+Ssa6oExBpTSmoZwyhBj9evubd2Kxu/GSKR7BwUIgstOgODX78Z3IKLHvY/ywL/4hiH/KuwsNTK2zhUZKg8p0Ty2nts5T1wk/O2gGHPQfd7nPUQlaC09b+eOj9Kd7NOoye4g55U8+beog/B1cFFeQh64ZX9HnxueqVeq/2x+bPFhY/V9uJ5OEpNq6TYSz1JrM4CajNfh+qsW++MBfIWw846v4N5EWkwA6Rc3qV4uRNppyhlLHKBjJcvdGWtVJcyr2za9NGM3T380IFlbGqNKjT62mTrzsTEVCPBvimfKSTU0fPc2Q3+kDsdqsYvos2WMjYc71fIBVxIE31RtleapjWGbaGqo7Ipbt+ATS6CCcYHVj0uV5XnfLNv8QLp7knY3ttGKnfSQbtwC1w1RacvoIKcYyi96DdPtsg+5oV5/leCQRsHqnCDJqUen4SsBlUGrlxNI7Y9eHdAtAjGGm2L34IFQfHwnzHIXWwyslXNCljg5XKGepmgsQgCF1fwSS/Me180GSZSQTtnHXQGfPVgyKN1/lWeV4bNnt+B++l91hImQqGlBUG/hlc2mgbZd5o9esYyq87f9Y59YAkbpFnJvdzqvW6ZUuUMR3L3LAWzrO3YHTx3sEYnz+v1fIlQKdGxxLBtNQn0HcZgGIqc/yNbPyDChju5PbET9VnXphW48ioYyPUOXE/TqsYgEGqDYuAkgNn9x6qJ36nG7rxXsci3YpDbM5h4Rgvh35U6aVIovYt/7ESJD9VMEJIohqIXs7kJOMookXvpMDIR/KMoIQiK9YTnwSKTst+zSAvU8KCQyiqlS/ihXFCkU/3KsCnYK615gVSGtX+3Oiyc+RvhA10klkY+3yCkPbO6+CGoVjCb5iCd7DIeoU0a1Tf0HGZUrFry1QX6kdiVDAME0+8Nsd5Tg9bYFvJ0MGaJMyhxKkUduxaGH9tlxiA3XYwi11ebaur0x7bxnhaa9NrIR2vPpyMmxmRUsp+UZ4SyDGmERAWS616z/t0v2+/sr0/xshJJnLTdxEphT1+32kXvX//m/wLqubtkWcLze',
          'yyssDDL': '1052',
          'languageCBL$0': 'on',
          'languageCBL$1': 'on',
          'languageCBL$2': 'on',
          'courseTB': 課程代碼,
          'coursenameTB': '',
          'instructorTB': '',
          'sessionTB': '',
          'sessionCBL$0': 'on',
          'sessionCBL$1': 'on',
          'sessionCBL$2': 'on',
          'sessionCBL$3': 'on',
          'sessionCBL$4': 'on',
          'sessionCBL$5': 'on',
          'sessionCBL$6': 'on',
          'sectionCBL$0': 'on',
          'sectionCBL$1': 'on',
          'sectionCBL$2': 'on',
          'sectionCBL$3': 'on',
          'statusCBL$0': 'on',
          'statusCBL$1': 'on',
          'statusCBL$2': 'on',
          'farCBL$0': 'on',
          'farCBL$1': 'on',
          'kerlCBL$0': 'on',
          'kerlCBL$1': 'on',
          'eligibleCBL$0': 'on',
          'eligibleCBL$1': 'on',
          'chargeCBL$0': 'on',
          'chargeCBL$1': 'on',
          'remainRBL': 'all',
          'numberpageRBL': '20',
          'lang': '1'
        }

        resp = resq.post('http://wa.nccu.edu.tw/QryTor/', data=data)
#         print(resp.text)
        data = {
          '__EVENTTARGET': '',
          '__EVENTARGUMENT': '',
          '__LASTFOCUS': '',
          '__VIEWSTATE': '/wEPDwULLTE1NDM5NTE1NjIPFgIeDWNvbnRyb2xzYWRkZWRnFgICAw9kFgICAQ9kFhoCAw8WAh4HVmlzaWJsZWhkAgcPFgIfAWhkAgsPDxYCHwFoZGQCEQ9kFgQCAQ8PFgIeBFRleHQFKOWFseaJvuWIsDHnrYbos4fmlpksMeizh+aWmeWPr+S+m+eAj+imvS5kZAIdDw8WAh8BaGRkAhUPEGRkFgECAWQCFw8WAh8BaBYCAgUPZBYCAiUPFgIfAWhkAkUPDxYCHgdFbmFibGVkaGRkAkcPDxYCHwNoZGQCSw8PFgIfA2hkZAJNDw8WAh8DaGRkAlEPEGRkFgECAWQCUw8PFgIfAgUMUGFnZTogMSBvZiAxZGQCVw8QZBAVAQExFQEBMRQrAwFnFgFmZBgBBR5fX0NvbnRyb2xzUmVxdWlyZVBvc3RCYWNrS2V5X18WAgUacXJ5cmVzdWx0MCR0cmFja2luZ19saXN0SUIFFXFyeXJlc3VsdDAkc2NobV90cGVJQjEZS4jRT2bBPK9eIGdUTwVp124bWTXwhDb8o35gkz7J',
          '__VIEWSTATEGENERATOR': '',
          '__SCROLLPOSITIONX': '0',
          '__SCROLLPOSITIONY': '0',
          'numberpageRBL': '20',
          'qryresult0$tracking_listIB.x': '10',
          'qryresult0$tracking_listIB.y': '3',
          'numberpageRBL2': '20'
        }

        resp = resq.post('http://wa.nccu.edu.tw/Qrytor/qryScheduleResult.aspx?language=zh-TW', data=data)
        html = resp.text
#         print(resp.text)
        page = etree.HTML(html)
        posturl = "http://wa.nccu.edu.tw/Qrytor/"+page.xpath("//form[@name='form1']/@action")[0]
        cookies = {
            '_ga': 'GA1.3.946599470.1481424474',
            'ASP.NET_SessionId': 'zyl0krhxudrhow35qngjq2qd',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
            'Connection': 'keep-alive',
            'Host': 'wa.nccu.edu.tw',
            'Referer': 'http://wa.nccu.edu.tw/QryTor/qryScheduleResult.aspx',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0',
            'Content-Type': 'application/x-www-form-urlencoded',
        }

        data = {
          '__EVENTTARGET': '',
          '__EVENTARGUMENT': '',
          '__VIEWSTATE': viewstate,
          '__VIEWSTATEGENERATOR': '221DE3B3',
          'LDAPTB': 使用者帳號,
          'passTB': 使用者密碼,
          'loginB': '登入'
        }

        resp = requests.post(posturl, data=data)
        resp = requests.post(posturl, data=data)
#         print(resp.text)
        print("加入成功囉！")
        url = 'http://wa.nccu.edu.tw/QryTor/goMySelSubList.aspx'
        f = myopener.open(url)
        soup = BeautifulSoup(f, "lxml")
        viewstate = soup.select("#__VIEWSTATE")[0]['value']
        data = {
          '__EVENTTARGET': '',
          '__EVENTARGUMENT': '',
          '__VIEWSTATE': viewstate,
          '__VIEWSTATEGENERATOR': '221DE3B3',
          'LDAPTB': 使用者帳號,
          'passTB': 使用者密碼,
          'loginB': '登入'
        }

        resp = requests.post('http://wa.nccu.edu.tw/QryTor/goMySelSubList.aspx', data=data)
        html = resp.text
        page = etree.HTML(html)
        print("你現在的清單裡有：")
        for i in page.xpath("//table[@id='tracking_listGV']//tbody//tr"):
            print(" ".join(i.xpath("./td[6]//text()")).split(" ")[0], " ".join(i.xpath("./td[5]//text()")).split(" ")[53], " ".join(i.xpath("./td[3]//text()")).split(" ")[53])


# 查看清單
def 查看清單(使用者帳號, 使用者密碼):
    class MyOpener(urllib.request.URLopener):
        version = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0'

    myopener = MyOpener()
    url = 'http://wa.nccu.edu.tw/QryTor/goMySelSubList.aspx'
    f = myopener.open(url)
    soup = BeautifulSoup(f, "lxml")
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    cookies = {
        '_ga': 'GA1.3.946599470.1481424474',
        'sIdx': '100015d6743e38ceb61fd2e4ff8511e5ee37e',
        'ASP.NET_SessionId': 'pm1nxmelcnrc5tbmk4g3srlw',
        '_gat': '1',
        '.LDAPAUTH': '537D074695A10AB697A649736892C889A02211ACCE822A2A8CBF75C6ACD504CBE35EF88DD2AC7CAE200A974FBE69D471C12C70127F40BC1218E440FEB76456AF5BCFD5DC7CCFB09DDCA6DFDDCFAC967BD55F5B435AA4701A451478E4CDDB3842B1378F34C79A39155C346C799B1C6EE42F1FC453A2B61EDA4C59A7854D2004AA83105918',
        'inccuLan': 'zh-TW',
        'NCCUWEBSSO': '93FF44A53CA0ED892B38817B95D550BDDDAF41D545E8ABB98DA6459BF2E86F5B4594A3F0AEC799DBB75565F66187A540F265CDD5670D4A942365EE057CCC8034F87786FEB9966B08A9F733EE3959A0A46FBC3CAED5A37739EDD5810BF0922B8C38DE855F43B2F2898040A3BB10A55D93CB3D69BEEC41CE0D05DC7D05A1786BB202EEDF28CE554BD1F1E660F5453B7410D3C931F7',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'Host': 'wa.nccu.edu.tw',
        'Referer': 'http://wa.nccu.edu.tw/QryTor/qryScheduleResult.aspx',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:50.0) Gecko/20100101 Firefox/50.0',
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    data = {
      '__EVENTTARGET': '',
      '__EVENTARGUMENT': '',
      '__VIEWSTATE': viewstate,
      '__VIEWSTATEGENERATOR': '221DE3B3',
      'LDAPTB': 使用者帳號,
      'passTB': 使用者密碼,
      'loginB': '登入'
    }

    resp = requests.post('http://wa.nccu.edu.tw/QryTor/goMySelSubList.aspx', data=data)
    html = resp.text
    page = etree.HTML(html)
    print("Aっ豆: 你現在的清單裡有：")
    for i in page.xpath("//table[@id='tracking_listGV']//tbody//tr"):
        print(" ".join(i.xpath("./td[6]//text()")).split(" ")[0], " ".join(i.xpath("./td[5]//text()")).split(" ")[53], " ".join(i.xpath("./td[3]//text()")).split(" ")[53])

def menu(name):
    print("Aっ豆: 我可以幫助你以下功能喔!")
    print("        1.輸入課程名稱或老師查課程")
    print("        2.可以進階查詢喔(可輸入星期或課程代碼)")
    print("        3.找到喜歡的課程後還可以加入清單!")
    print("        4.可以看看自己的清單~")
    print("Aっ豆: 請選擇~或打5離開~")
    i=input('%s: ' %(name))
    while(i!='5'):
        if(i=='1'):
            print("Aっ豆: 你要找的課程叫什麼？沒有的話就空白吧")
            課程名稱 = input('%s: ' %(name))
            print("Aっ豆: 你要找的課程教授是誰呀？沒有的話一樣空白就好")
            指導教授 = input('%s: ' %(name))
            searchCourse(課程名稱, 指導教授)
        elif(i=='2'):
            print("Aっ豆: 你要找的課程叫什麼？")
            課程名稱 = input('%s: ' %(name))
            print("Aっ豆: 你要找的課程教授是誰呀？")
            指導教授 = input('%s: ' %(name))
            print("Aっ豆: 你要找的課程知道代碼嗎？")
            課程代碼 = input('%s: ' %(name))
            print("Aっ豆: 那你要找哪幾天？給我數字或文字，如果要查禮拜天和禮拜三輸入'73'或'日三'都可以")
            星期 = input('%s: ' %(name))
            進階查詢(星期,課程代碼,課程名稱,指導教授)
        elif(i=='3'):
            print("Aっ豆: 請你告訴我你的帳號")
            使用者帳號 = input('%s: ' %(name))
            print("Aっ豆: 請你告訴我你的密碼")
            使用者密碼 = input('%s: ' %(name))
            print("Aっ豆: 請你告訴我想要加到清單的課程代碼")
            課程代碼 = input('%s: ' %(name))
            加入課程(使用者帳號, 使用者密碼, 課程代碼)
        elif(i=='4'):
            print("Aっ豆: 請你告訴我你的帳號")
            使用者帳號 = input('%s: ' %(name))
            print("Aっ豆: 請你告訴我你的密碼")
            使用者密碼 = input('%s: ' %(name))
            查看清單(使用者帳號, 使用者密碼)
        print("Aっ豆: 還要使用其他功能嗎?不需要可以打5~")
        i=input('%s: ' %(name))
    print("Aっ豆: 很開心幫到你～")

In [8]:
# 找關鍵字
def 找尋關鍵字(s):
    要去查的詞 = []
    關鍵字 = {}
    for x, w in jieba.analyse.extract_tags(s, withWeight=True):
        關鍵字[x] = w
    可能詞 = [k for k,v in sorted(關鍵字.items(), key=lambda item: item[1], reverse=True)]

    for i in [val for val in 可能詞 if val in 刪除關鍵詞庫]:
        可能詞.remove(i)
    if(len(可能詞)<=1):
        要去查的詞.append(可能詞[0])
        要去查的詞 = ""+要去查的詞[0]+""
    else:
        要去查的詞.append(可能詞[0])
        要去查的詞.append(可能詞[1])
        要去查的詞 = ""+要去查的詞[0]+" "+要去查的詞[1]+""
    return 要去查的詞

In [9]:
message = ''
name = input('%s: %s %s 你叫什麼名字呀？' %('Aっ豆', rn.choice(歡迎), 問好()))
print('%s: %s' %('Aっ豆', rn.choice(回歡迎)))
message = input('%s: ' %(name))
while not any(i in message for i in 離開):
    if any(i in message for i in 天氣):
        weather(name)
    elif any(i in message for i in 課程):
        menu(name)
    elif '課' in message:
        print('%s: %s %s' %('Aっ豆', rn.choice(敷衍), rn.choice(問選課)))
        message = input('%s: ' %(name))
        if any(i in message for i in 回選課):
            print('%s: %s %s' %('Aっ豆', rn.choice(敷衍), rn.choice(繼續)))
        else:
            menu(name)
    elif any(i in message for i in 吃東西):
        食物(name, 餐點)
    elif any(i in message for i in 喝東西):
        食物(name, 飲料)
    elif any(i in message for i in 時間):
        t = time.localtime(time.time())
        print('%s: 現在 %d年 %d月 %d日 星期%d %d點 %d分 %d秒 %s' %('Aっ豆', t[0], t[1], t[2], t[6]+1, t[3], t[4], t[5], rn.choice(繼續)))
    elif '翻譯' in message:
        翻譯(name)
    elif any(i in message for i in 感覺):
        print('%s: 我覺得%s %s' %('Aっ豆', rn.choice(回感覺), rn.choice(繼續)))
    elif any(i in message for i in 壞心情):
        print('%s: %s %s' %('Aっ豆', rn.choice(回壞心情), rn.choice(繼續)))
    elif any(i in message for i in 好心情):
        print('%s: %s %s' %('Aっ豆', rn.choice(回好心情), rn.choice(繼續)))
    elif any(i in message for i in 感謝):
        print('%s: %s %s' %('Aっ豆', rn.choice(回感謝), rn.choice(繼續)))
    elif any(i in message for i in 查詢):
        for i in 查詢:
            message = message.replace(i,"")
        x = 找尋關鍵字(message)
        print('%s: 你要找的是 (%s) 嗎？(y/n)' %('Aっ豆', x))
        ans = input('%s: ' %(name))
        if ans == 'y':
            msg = u''+x+''
        else:
            msg = u''+message+'' 
        encoded = msg.encode('utf8')
        x = repr(encoded)
#         s="https://www.google.com.tw/#q="+x.replace("b'","").replace("\\x","%").upper()[0:-1]
        s = "https://www.google.com.tw/search?q="+x.replace("b'","").replace("\\x","%").upper()[0:-1]+"&biw=1440&bih=714&dpr=1"
        a=webbrowser.get(''+webbrowser._tryorder[0]+'')
        a.open(s)
        print("Aっ豆: 查詢結果如何?")
    else:
        a = rn.choice([0, 1, 2, 3, 4, 5])
        if a < 5:
            print('%s: %s %s' %('Aっ豆', rn.choice(敷衍), rn.choice(繼續)))
        else:
            if(len(message)>5):
                x=找尋關鍵字(message)
                msg = u''+x+''
                encoded = msg.encode('utf8')
                x = repr(encoded)
            else:
                x=message
                msg = u''+x+''
                encoded = msg.encode('utf8')
                x = repr(encoded)
            print("Aっ豆: 我聽不太懂ㄟ！我幫你google一下好嗎？")
            message=input('%s: ' %(name))
            if ("不" not in message):      
#                 s="https://www.google.com.tw/#q="+x.replace("b'","").replace("\\x","%").upper()[0:-1]
                s = "https://www.google.com.tw/search?q="+x.replace("b'","").replace("\\x","%").upper()[0:-1]+"&biw=1440&bih=714&dpr=1"
                a=webbrowser.get(''+webbrowser._tryorder[0]+'')
                a.open(s)
                print("Aっ豆: 查詢結果如何?")
            else:
                print("Aっ豆: 好吧!")
    message = input('%s: ' %(name))
    
    
    
    
    
    

    
    
    
    
print('%s: %s %s' %('Aっ豆', rn.choice(回離開), name))

Aっ豆: 嗨！ 好晚了!怎麼還不快睡覺! 你叫什麼名字呀？月月
Aっ豆: 要聊什麼呀？
月月: 我肚子好餓，我想吃點東西
Aっ豆: 舉個例子吧？
月月: 炒飯！
Aっ豆: 可以參考 來來, 可以接受嗎？
月月: 不要
Aっ豆: 有特別喜歡什麼嗎？
月月: 改吃燉飯好了
Aっ豆: 可以參考 鄰居家, 可以嗎？
月月: 誒～好久沒吃了，謝謝啦
Aっ豆: 幫到你很開心 要再聊聊嗎？
月月: 我也想喝飲料
Aっ豆: 想要什麼種類呢？
月月: 奶茶
Aっ豆: 可以參考 谷庄茶奶, ＯＫ嗎？
月月: 不要
Aっ豆: 舉個例子吧？
月月: 還是奶茶就好
Aっ豆: 可以參考 路易莎, ＯＫ嗎？
月月: 噢！不錯誒
Aっ豆: 幫到你很開心 還有嗎？
月月: 現在幾點了呀？
Aっ豆: 現在 2017年 1月 8日 星期7 23點 1分 32秒 多講點
月月: 我想看一下選課，你能幫我嗎？
Aっ豆: 我可以幫助你以下功能喔!
        1.輸入課程名稱或老師查課程
        2.可以進階查詢喔(可輸入星期或課程代碼)
        3.找到喜歡的課程後還可以加入清單!
        4.可以看看自己的清單~
Aっ豆: 請選擇~或打5離開~
月月: 1
Aっ豆: 你要找的課程叫什麼？沒有的話就空白吧
月月: 數學
Aっ豆: 你要找的課程教授是誰呀？沒有的話一樣空白就好
月月: 


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:358: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods


Aっ豆: ㄟ豆...幫你找到這學期你查的課程只有10筆資料喔!
數學、邏輯與人生  曾睿彬  二78E  科目代碼是： 043030001
數學、邏輯與人生  蔡炎龍  三78E  科目代碼是： 043030011
服務學習課程－校園服務與數學諮詢  張宜武  二CD  科目代碼是： 050091001
數學領域教學實習  洪煌堯  二78  科目代碼是： 102118001
數學領域教學實習  洪煌堯  二78  科目代碼是： 109035001
工程數學  邱式鴻  五34C  科目代碼是： 207082002
財務數學（一）  黃泓智  一67  科目代碼是： 308878001
代數學  張宜武  二4四34  科目代碼是： 701010002
離散數學  李陽明  二1四12  科目代碼是： 701011002
數學軟體應用  蔡炎龍  一78E  科目代碼是： 701772002
Aっ豆: 還要使用其他功能嗎?不需要可以打5~
月月: 4
Aっ豆: 請你告訴我你的帳號
月月: 102701014
Aっ豆: 請你告訴我你的密碼
月月: test123


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:382: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods


Aっ豆: 你現在的清單裡有：
五234 沈錳坤 演算法
Aっ豆: 還要使用其他功能嗎?不需要可以打5~
月月: 2
Aっ豆: 你要找的課程叫什麼？
月月: 
Aっ豆: 你要找的課程教授是誰呀？
月月: 
Aっ豆: 你要找的課程知道代碼嗎？
月月: 701772002
Aっ豆: 那你要找哪幾天？給我數字或文字，如果要查禮拜天和禮拜三輸入'73'或'日三'都可以
月月: 1
ㄟ豆...幫你找到這學期你查的課程只有 1筆資料喔!
數學軟體應用  蔡炎龍  一78E  科目代碼是： 701772002
Aっ豆: 還要使用其他功能嗎?不需要可以打5~
月月: 3
Aっ豆: 請你告訴我你的帳號
月月: 102701014
Aっ豆: 請你告訴我你的密碼
月月: test123
Aっ豆: 請你告訴我想要加到清單的課程代碼
月月: 701772002


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:376: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods


加入成功囉！
你現在的清單裡有：
一78E 蔡炎龍 數學軟體應用
五234 沈錳坤 演算法
Aっ豆: 還要使用其他功能嗎?不需要可以打5~
月月: 5
Aっ豆: 很開心幫到你～
月月: 可以幫我google一下蔡炎龍嗎


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jc/hp3pssbx23s2ggf_9qd7_dqm0000gn/T/jieba.cache
Loading model cost 1.383 seconds.
Prefix dict has been built succesfully.


Aっ豆: 你要找的是 (蔡炎龍 一下) 嗎？(y/n)
月月: n
Aっ豆: 查詢結果如何?
月月: 幫我查ppap
Aっ豆: 你要找的是 (ppap) 嗎？(y/n)
月月: y
Aっ豆: 查詢結果如何?
月月: 不錯，謝謝你
Aっ豆: 能幫到你很開心！ 
月月: 今天天氣如何呀？
Aっ豆: 查詢中。。。
Aっ豆: 這是我幫你查的天氣！
         現在溫度 臺北市 : 18 度
         今日最高溫為 21 度
         今日最低溫為 18 度
         今日下雨機率 : 5%
        天氣不錯呢!
Aっ豆: 要進入網頁嗎?
月月: 要
Aっ豆 :知道天氣就可以安心出門囉!
月月: 我剛剛看要一點不會的東西，你可以幫我翻譯嗎？
Aっ豆: 你想翻譯哪兩國語言呢？
       若想將中文翻譯成英文，請輸入：中文，英文
月月:中文，英文
Aっ豆: 你想要翻譯什麼？
月月:蜻蜓
Aっ豆: 翻譯中。。。
Aっ豆: dragonfly
Aっ豆: 還要繼續嗎？
月月: 要
Aっ豆: 你想翻譯哪兩國語言呢？
       若想將中文翻譯成英文，請輸入：中文，英文
月月:英文，英文
Aっ豆: 兩個一樣囉！
Aっ豆: 你想翻譯哪兩國語言呢？
       若想將中文翻譯成英文，請輸入：中文，英文
月月:中文，日文
Aっ豆: 你想要翻譯什麼？
月月:蔬菜
Aっ豆: 翻譯中。。。
Aっ豆: 野菜
Aっ豆: 還要繼續嗎？
月月: 不用了，謝謝你
Aっ豆: 很開心幫到你！
月月: 有點累了，今天先這樣好了
Aっ豆: 好吧！ 還有嗎？
月月: 沒了，先下線囉！掰掰
Aっ豆: 很高興認識你！ 月月
